# Short-Term Synaptic Plasticity

## Intro
Short-term synaptic plasticity (or short-term plasticity [STP]) is a phenomenon in which synaptic efficacy changes over time in response to previous presynaptic occurances.

Two types of STP have been observed:

* **Short-Term Depression** (STD)
* **Short-Term Facilitation** (STF)

Short-Term Depression is caused by depletion of neurotransmitters at the axon terminal while Short-Term Facilitation is caused by calcium influx into the axon terminal after a spike. This increases the release probablility of neurotransmitters.

STP typically occurs on the order of hundereds to thousands of milliseconds and is temporary. Without activity the synapse will quickly return to basline levels.

STP may serve as a substrate for processing temporal information. The response of a post-synaptic neuron depends on the history of the presynaptic activity. This information could theoretically be extracted for use. STP can enrich computational studies, allowing networks to produce capabilities difficult to be seen using static connections.

## Determining STD/STF

A probabilistic model is used to model these special synapses. The model used in this notebook is based on the the paper found [here.](https://journals.physiology.org/doi/full/10.1152/jn.00258.2001)

Synaptic connections are made up of a number of sites, each with a max of one vesicle that can be released. Given a presynaptic spike at time $t_{sp}$, sites with a vesicle will release it with probability $U_{SE}$. After a release, the site can be refilled at any time interval $dt$ with a probability of $\frac{dt}{\tau_{rec}}$. These can be combined into a single differential equation detailing how the probability $P_{v}$ for a site to contain a vesicle with respect to any time $t$ changes as shown below:

$$
\frac{dP_{v}}{dt} = \frac{(1-P_{v})}{\tau_{rec}}-U_{SE} * P_{v} * \delta(t-t_{sp})
$$

Given the probability, $P_{v}$, that a vesicle is in a site, and the probability, $U_{SE}$, that a vesicle will be released, the probabiliity of release, $P_{r}$, at the time of a spike, $t_{sp}$, can be expressed in the equation below:
$$
P_{r}(t_{sp}) = U_{SE} * P_{v}
$$




Within depressive synapses(STD), $U_{SE}$ is constant. In facilitative synapses(STF), $U_{SE}$ is given by its own differential equation that changes with respect to time $t$:
$$
\frac{dU_{SE}}{dt} = -\frac{U_{SE}}{\tau_{facil}} + U1 * (1-U_{SE}) * \delta(t-t_{sp})
$$
where $\tau_{facil}$ is the relaxation time constant of facilitation and $U1$ is a constant determining the step increase in $U_{SE}$.

In [2]:
import os

RunningInCOLAB = 'google.colab' in str(get_ipython())  # checks to see if we are in google colab
if RunningInCOLAB:                                     # installs packages and repo if in colab
    !pip install ipywidgets &> /dev/null               
    !pip install neuron &> /dev/null

    !git clone https://github.com/GregGlickert/Neural-Modeling-Manual.git &> /dev/null 
    %cd Neural-Modeling-Manual/Chapter-3-Synaptic-Design/Section-2-Short-Term-Plasticity/

if(os.path.exists('x86_64')==False):        
    os.system('nrnivmodl modfiles') # compile modfiles. Return 0 for success, 1 for failure.

## Interactive Model
Below is a model with widgets that allow testing of this version of dynamic synapses. Given is the ability to change the frequency of the tonic input spike train, the weight of the synapse, the initial efficacy, or initial value for $U_{SE}$, and the time constants for the two processes are given. The two buttons give example values used in the paper for their facilitative and depressive synapses.

In [32]:
#@title Run Cell to activate model
from neuron import h
from neuron.units import ms, mV
import matplotlib.pyplot as plt
from matplotlib import gridspec
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

def model(weight,input_freqency,Use,tau_f,tau_d):
    h.load_file('stdrun.hoc')
    h.load_file('PN_cells.hoc') # reads in template
    cell = h.PN_C() #cell from template file
    h.dt = 0.1 # time step (resolution) of the simulation in ms
    h.tstop = 2000 # how long to run the simulation in ms
    h.v_init= -70 # initial membrane potential in mV
    
    t = h.Vector()      # Vectors are used to record data in the simulation
    soma_v = h.Vector()
    i = h.Vector()
    g = h.Vector()
    g_AMPA = h.Vector()
    g_NMDA = h.Vector()
    
    
    conn = h.AMPA_NMDA_STP(cell.dend[0](0.9)) # makes a syn connected to soma
    conn.initW = weight #synaptic weight for our synapse

    conn.Fac = tau_f # faciliation parameter
    conn.Dep = tau_d # depression parameter
    conn.Use = Use   # inital efficacy


    nstim = h.NetStim(0.9)               # creating a stimulation for the cell

    nstim.interval=1000/input_freqency   # Hz of input
    nstim.start=500                      # when stim should start
    nstim.number = 100                   # 
    nstim.noise = 0                      # level of noise
    nc = h.NetCon(nstim,conn,10,10,1)
        
    
    t.record(h._ref_t) # derefences and records data
    soma_v.record(cell.soma[0](0.5)._ref_v)
    i.record(conn._ref_i)
    g.record(conn._ref_g)
    g_AMPA.record(conn._ref_g_AMPA)
    g_NMDA.record(conn._ref_g_NMDA)


    h.finitialize(h.v_init * mV)  # runs sims
    h.continuerun(h.tstop * ms)   
    
    plot_model(t,soma_v,i)
    
def plot_model(t,soma_v,i):         # ploting code    
    fig = plt.figure(figsize=(8, 8),tight_layout=True) 
    gs = gridspec.GridSpec(2, 1, width_ratios=[1],height_ratios=[1,1]) 
    
    ax0 = plt.subplot(gs[0,0])  # top plot
    ax1 = plt.subplot(gs[1,0])  # bottom plot
    
    ax0.plot(t ,soma_v)
    ax0.set_title('Soma Voltage')
    ax0.set_ylabel('Voltage(mV)')
    
    ax1.plot(t,i)
    ax1.set_title('Synaptic Current')
    ax1.set_ylabel('Current(nA)')

    
    #ax2.plot(t,Pr)
    #ax2.set_title("running synaptic efficacy")
    


    plt.show()


    
weight = 14         # inital values used by the slider
input_freqency = 7  
Use = 1
tau_f = 10
tau_d = 100

# widgets the get displayed below
w_run = widgets.Button(description='Run',icon='history',button_style='primary')
w_dep_model = widgets.Button(description='Depressing syn',icon='history',button_style='')    
w_fac_model = widgets.Button(description='Faciliating syn',icon='history',button_style='')        
w_weight = widgets.FloatSlider(value=weight,min=0.001,max=50, step=0.2,description='Syn. Weight',style=dict(description_width='initial'))
w_input_freqency = widgets.FloatSlider(value=input_freqency,min=0.001,max=50, step=1,description='Input Freq.',style=dict(description_width='initial'))
w_Use = widgets.FloatSlider(value=Use,min=0.001,max=1, step=0.01,description='Init. Efficacy',style=dict(description_width='initial'))
w_tau_f = widgets.FloatSlider(value=tau_f,min=0,max=2000, step=10,description='Tau Fac.',style=dict(description_width='initial'))
w_tau_d = widgets.FloatSlider(value=tau_d,min=0,max=2000, step=10,description='Tau Dep.',style=dict(description_width='initial'))

def run_model(*arges):
    clear_output()  #               Removes the sliders and then updated the values and displays the new valuesgit
    
    weight = w_weight.value
    input_freqency = w_input_freqency.value
    Use = w_Use.value
    tau_f = w_tau_f.value
    tau_d = w_tau_d.value
    display(ui)
    model(weight,input_freqency,Use,tau_f,tau_d)

def depressing_model(*arges):
    clear_output()
    print("changing synaptic values!")
    w_Use.value = 0.5 #based on the paper
    w_tau_d.value = 800
    w_tau_f.value = 0
    display(ui)

def fac_model(*arges):
    clear_output()
    print("changing synaptic values!")
    w_Use.value = 0.03 #based on the paper
    w_tau_d.value = 300
    w_tau_f.value = 1800
    display(ui)

w_run.on_click(run_model)
w_dep_model.on_click(depressing_model)
w_fac_model.on_click(fac_model) 
ui = VBox([HBox([w_run,w_dep_model,w_fac_model,w_Use]), HBox([w_weight,w_tau_f]),HBox([w_input_freqency,w_tau_d])])
ui

## Exercises
Given these plots of synaptic current and voltage, tune this model to give the same results.
# Add plots of synapses